In [2]:
import pandas as pd
import numpy as np 
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle

In [3]:
interfaces_data = pd.read_csv("/qfs/projects/proteometer/pheno_analysis/results/merged_interfaces_FINAL2.csv")
pockets_data = pd.read_csv("/qfs/projects/proteometer/pheno_analysis/results/merged_pockets_FINAL.csv")
stability_data = pd.read_csv("/qfs/projects/proteometer/pheno_analysis/results/merged_stability_FINAL.csv")

In [17]:
interfaces_data

,Unnamed: 0.1,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,...,uniprotID,AA,res_number,pK,state,position,uniprot_id,closest_interface,inside_interface,distance_from_interface
0,0,0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,...,P31946,THR,2.0,NaN,undefined,2.0,P31946,P51114,0,18.400596
1,1,1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,...,P31946,SER,6.0,NaN,undefined,6.0,P31946,",P31946",1,0.000000
2,2,2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,...,P31946,TYR,21.0,11.100927,protonated,21.0,P31946,",P31040,P30086,P12931,P67775,P68402,O75317,Q3L...",1,0.000000
3,3,3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,...,P31946,THR,32.0,NaN,undefined,32.0,P31946,O75317,0,15.828202
4,4,4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,...,P31946,SER,39.0,NaN,undefined,39.0,P31946,Q9UHV9,0,16.123331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436007,435125,435258,PRAP1,PRAP1,Q96NZ9,10q26.3,Y113-ga,27914389,human,17.21,...,Q96NZ9,TYR,113.0,9.888425,protonated,113.0,Q96NZ9,Q96EQ0,0,19.475904
436008,435127,435260,PRG2,PRG2,P13727,11q12.1,T23-ga,27914778,human,25.21,...,P13727,THR,23.0,NaN,undefined,23.0,P13727,Q86Z23,0,25.513050
436009,435128,435261,PRG2,PRG2,P13727,11q12.1,S24-ga,27914781,human,25.21,...,P13727,SER,24.0,NaN,undefined,24.0,P13727,Q86Z23,0,27.206643
436010,435129,435262,PRG2,PRG2,P13727,11q12.1,T25-ga,27914784,human,25.21,...,P13727,THR,25.0,NaN,undefined,25.0,P13727,Q86Z23,0,29.210871


In [18]:
pockets_data

,Unnamed: 0.1,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,...,uniprotID,AA,res_number,pK,state,position,uniprot_id,closest_pocket,inside_pocket,distance_from_pocket
0,0,0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,...,P31946,THR,2.0,NaN,undefined,2.0,P31946,2,0,22.103256
1,1,1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,...,P31946,SER,6.0,NaN,undefined,6.0,P31946,10,0,18.407673
2,2,2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,...,P31946,TYR,21.0,11.100927,protonated,21.0,P31946,",2,15",1,0.000000
3,3,3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,...,P31946,THR,32.0,NaN,undefined,32.0,P31946,10,0,12.921682
4,4,4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,...,P31946,SER,39.0,NaN,undefined,39.0,P31946,",10",1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436007,435125,435258,PRAP1,PRAP1,Q96NZ9,10q26.3,Y113-ga,27914389,human,17.21,...,Q96NZ9,TYR,113.0,9.888425,protonated,113.0,Q96NZ9,1,0,22.441628
436008,435127,435260,PRG2,PRG2,P13727,11q12.1,T23-ga,27914778,human,25.21,...,P13727,THR,23.0,NaN,undefined,23.0,P13727,4,0,26.709694
436009,435128,435261,PRG2,PRG2,P13727,11q12.1,S24-ga,27914781,human,25.21,...,P13727,SER,24.0,NaN,undefined,24.0,P13727,4,0,29.435677
436010,435129,435262,PRG2,PRG2,P13727,11q12.1,T25-ga,27914784,human,25.21,...,P13727,THR,25.0,NaN,undefined,25.0,P13727,4,0,30.474608


In [19]:
stability_data

,Unnamed: 0.1,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,...,res_number,pK,state,position,uniprot_id,plddt,foldx_ddg_min,foldx_ddg_max,foldx_ddg_abs_max,foldx_ddg_abs_median
0,0,0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,...,2.0,NaN,undefined,2.0,P31946,74.30,-0.700058,0.130120,0.700058,0.096524
1,1,1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,...,6.0,NaN,undefined,6.0,P31946,95.62,-0.688997,0.643053,0.688997,0.278294
2,2,2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,...,21.0,11.100927,protonated,21.0,P31946,98.43,0.250080,3.602560,3.602560,2.686420
3,3,3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,...,32.0,NaN,undefined,32.0,P31946,98.24,-2.085020,19.898600,19.898600,3.844370
4,4,4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,...,39.0,NaN,undefined,39.0,P31946,97.05,-1.610100,2.446650,2.446650,1.516780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435870,435125,435258,PRAP1,PRAP1,Q96NZ9,10q26.3,Y113-ga,27914389,human,17.21,...,113.0,9.888425,protonated,113.0,Q96NZ9,87.73,-0.310755,0.859067,0.859067,0.516341
435871,435127,435260,PRG2,PRG2,P13727,11q12.1,T23-ga,27914778,human,25.21,...,23.0,NaN,undefined,23.0,P13727,39.53,-0.312937,0.381937,0.381937,0.134005
435872,435128,435261,PRG2,PRG2,P13727,11q12.1,S24-ga,27914781,human,25.21,...,24.0,NaN,undefined,24.0,P13727,45.31,-1.365570,0.171909,1.365570,0.471913
435873,435129,435262,PRG2,PRG2,P13727,11q12.1,T25-ga,27914784,human,25.21,...,25.0,NaN,undefined,25.0,P13727,40.84,-0.118950,0.782394,0.782394,0.100066


In [20]:
print(len(interfaces_data))
print(len(pockets_data))
print(len(stability_data))

436012
436012
435875


In [21]:
missing_stability = interfaces_data[~interfaces_data.SITE_GRP_ID.isin(stability_data.SITE_GRP_ID)]
missing_stability

,Unnamed: 0.1,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,...,uniprotID,AA,res_number,pK,state,position,uniprot_id,closest_interface,inside_interface,distance_from_interface
28,272398,272506,YWHAB,14-3-3 beta,P31946,20q13.12,K5-ac,33347661,human,28.08,...,P31946,LYS,5.0,12.177229,protonated,5.0,P31946,P51114,0,14.319542
29,272399,272507,YWHAB,14-3-3 beta,P31946,20q13.12,K13-ac,36297548,human,28.08,...,P31946,LYS,13.0,11.797996,protonated,13.0,P31946,P51114,0,3.847883
30,272400,272508,YWHAB,14-3-3 beta,P31946,20q13.12,K51-ac,474927,human,28.08,...,P31946,LYS,51.0,10.448943,protonated,51.0,P31946,",P05455,Q8ND76,Q6NUK4,P12931,P51965,Q676U5,Q9Y...",1,0.000000
31,272401,272509,YWHAB,14-3-3 beta,P31946,20q13.12,K70-ac,461819,human,28.08,...,P31946,LYS,70.0,10.624529,protonated,70.0,P31946,",O00422,Q9Y4H4,Q9P0L2,P62314,Q9H9C1,Q6WKZ4",1,0.000000
32,272402,272510,YWHAB,14-3-3 beta,P31946,20q13.12,K77-ac,31068005,human,28.08,...,P31946,LYS,77.0,10.236019,protonated,77.0,P31946,",P61981,Q04917,P27348,P62258,P63104,P31946,P31...",1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,297998,298115,SFN,14-3-3 sigma,P31947,1p36.11,K109-ub,15383715,human,27.77,...,P31947,LYS,109.0,10.921604,protonated,109.0,P31947,P63173,0,14.532466
233,297999,298116,SFN,14-3-3 sigma,P31947,1p36.11,K122-ub,3933061,human,27.77,...,P31947,LYS,122.0,11.556411,protonated,122.0,P31947,",P48730,O75494,Q6NUK4,Q9H6H4,P38159,Q14103,Q7Z...",1,0.000000
234,298000,298117,SFN,14-3-3 sigma,P31947,1p36.11,K124-ub,12381735,human,27.77,...,P31947,LYS,124.0,12.626807,protonated,124.0,P31947,",P63173",1,0.000000
235,298001,298118,SFN,14-3-3 sigma,P31947,1p36.11,K140-ub,964495106,human,27.77,...,P31947,LYS,140.0,11.151598,protonated,140.0,P31947,P63173,0,14.858370


In [37]:
missing_stability['uniprot_id'].unique()

array(['P31946', 'P62258', 'Q04917', 'P61981', 'P31947'], dtype=object)

In [25]:
with open("/qfs/projects/proteometer/pheno_analysis/stability_pickle_files/P31946_stability.pkl", 'rb') as handle:
        psp_only = pickle.load(handle)

psp_only

,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,...,res_number,pK,state,position,uniprot_id,plddt,foldx_ddg_min,foldx_ddg_max,foldx_ddg_abs_max,foldx_ddg_abs_median
0,0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,...,2,NaN,undefined,2.0,P31946,74.30,-0.700058,0.130120,0.700058,0.096524
1,1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,...,6,NaN,undefined,6.0,P31946,95.62,-0.688997,0.643053,0.688997,0.278294
2,2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,...,21,11.100927,protonated,21.0,P31946,98.43,0.250080,3.602560,3.602560,2.686420
3,3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,...,32,NaN,undefined,32.0,P31946,98.24,-2.085020,19.898600,19.898600,3.844370
4,4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,...,39,NaN,undefined,39.0,P31946,97.05,-1.610100,2.446650,2.446650,1.516780
5,5,YWHAB,14-3-3 beta,P31946,20q13.12,S47-p,4773271,human,28.08,14-3-3,...,47,NaN,undefined,47.0,P31946,98.26,-2.341240,5.251210,5.251210,1.085610
6,6,YWHAB,14-3-3 beta,P31946,20q13.12,Y50-p,12432961,human,28.08,14-3-3,...,50,NaN,undefined,50.0,P31946,98.70,-1.046360,5.260500,5.260500,2.518650
7,7,YWHAB,14-3-3 beta,P31946,20q13.12,S59-p,4935095,human,28.08,14-3-3,...,59,NaN,undefined,59.0,P31946,98.12,-0.641476,3.564730,3.564730,0.280184
8,8,YWHAB,14-3-3 beta,P31946,20q13.12,S60-p,450226,human,28.08,14-3-3,...,60,NaN,undefined,60.0,P31946,98.15,-2.040630,2.278820,2.278820,0.706567
9,9,YWHAB,14-3-3 beta,P31946,20q13.12,S66-p,15718715,human,28.08,14-3-3,...,66,NaN,undefined,66.0,P31946,93.63,-0.675410,3.533310,3.533310,0.284937


In [27]:
P31946 = missing_stability[missing_stability["ACC_ID"]=="P31946"]
P31946


,Unnamed: 0.1,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,...,uniprotID,AA,res_number,pK,state,position,uniprot_id,closest_interface,inside_interface,distance_from_interface
28,272398,272506,YWHAB,14-3-3 beta,P31946,20q13.12,K5-ac,33347661,human,28.08,...,P31946,LYS,5.0,12.177229,protonated,5.0,P31946,P51114,0,14.319542
29,272399,272507,YWHAB,14-3-3 beta,P31946,20q13.12,K13-ac,36297548,human,28.08,...,P31946,LYS,13.0,11.797996,protonated,13.0,P31946,P51114,0,3.847883
30,272400,272508,YWHAB,14-3-3 beta,P31946,20q13.12,K51-ac,474927,human,28.08,...,P31946,LYS,51.0,10.448943,protonated,51.0,P31946,",P05455,Q8ND76,Q6NUK4,P12931,P51965,Q676U5,Q9Y...",1,0.000000
31,272401,272509,YWHAB,14-3-3 beta,P31946,20q13.12,K70-ac,461819,human,28.08,...,P31946,LYS,70.0,10.624529,protonated,70.0,P31946,",O00422,Q9Y4H4,Q9P0L2,P62314,Q9H9C1,Q6WKZ4",1,0.000000
32,272402,272510,YWHAB,14-3-3 beta,P31946,20q13.12,K77-ac,31068005,human,28.08,...,P31946,LYS,77.0,10.236019,protonated,77.0,P31946,",P61981,Q04917,P27348,P62258,P63104,P31946,P31...",1,0.000000
33,272403,272511,YWHAB,14-3-3 beta,P31946,20q13.12,K82-ac,89113000,human,28.08,...,P31946,LYS,82.0,11.369275,protonated,82.0,P31946,Q9H3P2,0,7.715907
34,272404,272512,YWHAB,14-3-3 beta,P31946,20q13.12,K117-ac,466016,human,28.08,...,P31946,LYS,117.0,11.224421,protonated,117.0,P31946,Q6P597,0,16.795746
35,272405,272513,YWHAB,14-3-3 beta,P31946,20q13.12,K122-ac,3649439,human,28.08,...,P31946,LYS,122.0,11.394938,protonated,122.0,P31946,",P05455,O14757,Q8ND76,Q6NUK4,P51965,Q6P597,Q9Y...",1,0.000000
36,272406,272514,YWHAB,14-3-3 beta,P31946,20q13.12,K159-ac,12423074,human,28.08,...,P31946,LYS,159.0,10.978137,protonated,159.0,P31946,",P05455",1,0.000000
37,272407,272515,YWHAB,14-3-3 beta,P31946,20q13.12,K214-ac,38507636,human,28.08,...,P31946,LYS,214.0,10.722947,protonated,214.0,P31946,",Q8ND76,Q8WXF0,Q15287,Q92783,P49760,Q9NXR1,Q13...",1,0.000000


In [32]:
P31946['res_number'].values

array([  5.,  13.,  51.,  70.,  77.,  82., 117., 122., 159., 214.,   5.,
        11.,  13.,  29.,  51.,  70.,  82.,  87., 105., 117., 122., 124.,
       140., 159., 160., 195.,  51.])

In [33]:
psp_only['res_number'].values

array([  2,   6,  21,  32,  39,  47,  50,  59,  60,  66, 106, 116, 120,
       127, 130, 132, 136, 151, 180, 186, 207, 212, 213, 216, 217, 228,
       231, 232])

In [34]:
psp_data = pd.read_csv("/people/imal967/git_repos/pheno_analysis/phosphosite_for_pockets.csv")
psp_data

/tmp/ipykernel_10022/3773762598.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  psp_data = pd.read_csv("/people/imal967/git_repos/pheno_analysis/phosphosite_for_pockets.csv")


,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,...,Ambiguous_Site,RES_NUM,PKA_ID,uniprotID,AA,res_number,pK,state,position,uniprot_id
0,0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,...,0,2,P31946_2,P31946,THR,2.0,NaN,undefined,2.0,P31946
1,1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,...,0,6,P31946_6,P31946,SER,6.0,NaN,undefined,6.0,P31946
2,2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,...,0,21,P31946_21,P31946,TYR,21.0,11.100927,protonated,21.0,P31946
3,3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,...,0,32,P31946_32,P31946,THR,32.0,NaN,undefined,32.0,P31946
4,4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,...,0,39,P31946_39,P31946,SER,39.0,NaN,undefined,39.0,P31946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436007,436140,ZHX1,ZHX1,Q9UKY1,8q24.13,S450-gl,14703720,human,98.10,NaN,...,0,450,Q9UKY1_450,Q9UKY1,SER,450.0,NaN,undefined,450.0,Q9UKY1
436008,436141,ZNF281,ZNF281,Q9Y2X9,1q32.1,S891-gl,14703723,human,96.91,NaN,...,0,891,Q9Y2X9_891,Q9Y2X9,SER,891.0,NaN,undefined,891.0,Q9Y2X9
436009,436142,ZNF609,ZNF609,O15014,15q22.31,S1196-gl,14703726,human,151.19,NaN,...,0,1196,O15014_1196,O15014,SER,1196.0,NaN,undefined,1196.0,O15014
436010,436143,ZYX,Zyxin,Q15942,7q34,S169-gl,1876610700,human,61.28,NaN,...,0,169,Q15942_169,Q15942,SER,169.0,NaN,undefined,169.0,Q15942


In [36]:
psp_data[psp_data['uniprot_id'] == "P31946"]

,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,...,Ambiguous_Site,RES_NUM,PKA_ID,uniprotID,AA,res_number,pK,state,position,uniprot_id
0,0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,...,0,2,P31946_2,P31946,THR,2.0,NaN,undefined,2.0,P31946
1,1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,...,0,6,P31946_6,P31946,SER,6.0,NaN,undefined,6.0,P31946
2,2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,...,0,21,P31946_21,P31946,TYR,21.0,11.100927,protonated,21.0,P31946
3,3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,...,0,32,P31946_32,P31946,THR,32.0,NaN,undefined,32.0,P31946
4,4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,...,0,39,P31946_39,P31946,SER,39.0,NaN,undefined,39.0,P31946
5,5,YWHAB,14-3-3 beta,P31946,20q13.12,S47-p,4773271,human,28.08,14-3-3,...,0,47,P31946_47,P31946,SER,47.0,NaN,undefined,47.0,P31946
6,6,YWHAB,14-3-3 beta,P31946,20q13.12,Y50-p,12432961,human,28.08,14-3-3,...,0,50,P31946_50,P31946,TYR,50.0,NaN,undefined,50.0,P31946
7,7,YWHAB,14-3-3 beta,P31946,20q13.12,S59-p,4935095,human,28.08,14-3-3,...,0,59,P31946_59,P31946,SER,59.0,NaN,undefined,59.0,P31946
8,8,YWHAB,14-3-3 beta,P31946,20q13.12,S60-p,450226,human,28.08,14-3-3,...,0,60,P31946_60,P31946,SER,60.0,NaN,undefined,60.0,P31946
9,9,YWHAB,14-3-3 beta,P31946,20q13.12,S66-p,15718715,human,28.08,14-3-3,...,0,66,P31946_66,P31946,SER,66.0,NaN,undefined,66.0,P31946


In [9]:
pockets_data['SITE_GRP_ID'].nunique()

384304

In [10]:
### give every row a unique ID
# id's are assigned based on row # of original psp data

interfaces_data['uniqueID'] = interfaces_data['Unnamed: 0'].astype(str)
pockets_data['uniqueID'] = pockets_data['Unnamed: 0'].astype(str)
stability_data['uniqueID'] = stability_data['Unnamed: 0'].astype(str)


In [11]:
pockets_data['uniqueID'].nunique()

436012

In [12]:
merged_interface_pockets = pd.merge(interfaces_data, pockets_data[['uniqueID', 'closest_pocket', 'inside_pocket', 'distance_from_pocket']], how= 'inner', on= 'uniqueID')
merged_interface_pockets

,Unnamed: 0.1,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,...,state,position,uniprot_id,closest_interface,inside_interface,distance_from_interface,uniqueID,closest_pocket,inside_pocket,distance_from_pocket
0,0,0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,...,undefined,2.0,P31946,P51114,0,18.400596,0,2,0,22.103256
1,1,1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,...,undefined,6.0,P31946,",P31946",1,0.000000,1,10,0,18.407673
2,2,2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,...,protonated,21.0,P31946,",P31040,P30086,P12931,P67775,P68402,O75317,Q3L...",1,0.000000,2,",2,15",1,0.000000
3,3,3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,...,undefined,32.0,P31946,O75317,0,15.828202,3,10,0,12.921682
4,4,4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,...,undefined,39.0,P31946,Q9UHV9,0,16.123331,4,",10",1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436007,435125,435258,PRAP1,PRAP1,Q96NZ9,10q26.3,Y113-ga,27914389,human,17.21,...,protonated,113.0,Q96NZ9,Q96EQ0,0,19.475904,435258,1,0,22.441628
436008,435127,435260,PRG2,PRG2,P13727,11q12.1,T23-ga,27914778,human,25.21,...,undefined,23.0,P13727,Q86Z23,0,25.513050,435260,4,0,26.709694
436009,435128,435261,PRG2,PRG2,P13727,11q12.1,S24-ga,27914781,human,25.21,...,undefined,24.0,P13727,Q86Z23,0,27.206643,435261,4,0,29.435677
436010,435129,435262,PRG2,PRG2,P13727,11q12.1,T25-ga,27914784,human,25.21,...,undefined,25.0,P13727,Q86Z23,0,29.210871,435262,4,0,30.474608


In [13]:
merged_interface_pocket_stability = pd.merge(merged_interface_pockets, stability_data[['uniqueID', 'foldx_ddg_min', 'foldx_ddg_max', 'foldx_ddg_abs_max','foldx_ddg_abs_median']], how= 'inner', on= 'uniqueID')
merged_interface_pocket_stability

,Unnamed: 0.1,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,...,inside_interface,distance_from_interface,uniqueID,closest_pocket,inside_pocket,distance_from_pocket,foldx_ddg_min,foldx_ddg_max,foldx_ddg_abs_max,foldx_ddg_abs_median
0,0,0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,...,0,18.400596,0,2,0,22.103256,-0.700058,0.130120,0.700058,0.096524
1,1,1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,...,1,0.000000,1,10,0,18.407673,-0.688997,0.643053,0.688997,0.278294
2,2,2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,...,1,0.000000,2,",2,15",1,0.000000,0.250080,3.602560,3.602560,2.686420
3,3,3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,...,0,15.828202,3,10,0,12.921682,-2.085020,19.898600,19.898600,3.844370
4,4,4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,...,0,16.123331,4,",10",1,0.000000,-1.610100,2.446650,2.446650,1.516780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435870,435125,435258,PRAP1,PRAP1,Q96NZ9,10q26.3,Y113-ga,27914389,human,17.21,...,0,19.475904,435258,1,0,22.441628,-0.310755,0.859067,0.859067,0.516341
435871,435127,435260,PRG2,PRG2,P13727,11q12.1,T23-ga,27914778,human,25.21,...,0,25.513050,435260,4,0,26.709694,-0.312937,0.381937,0.381937,0.134005
435872,435128,435261,PRG2,PRG2,P13727,11q12.1,S24-ga,27914781,human,25.21,...,0,27.206643,435261,4,0,29.435677,-1.365570,0.171909,1.365570,0.471913
435873,435129,435262,PRG2,PRG2,P13727,11q12.1,T25-ga,27914784,human,25.21,...,0,29.210871,435262,4,0,30.474608,-0.118950,0.782394,0.782394,0.100066


In [14]:
merged_interface_pocket_stability.to_csv("/qfs/projects/proteometer/pheno_analysis/results/merged_interface_pocket_stability.csv")

In [1]:
435875 + 237

436112

In [5]:
stability_data = pd.read_csv("/qfs/projects/proteometer/pheno_analysis/results/merged_stability_FINAL.csv")
missing_data = pd.read_csv("/people/imal967/git_repos/pheno_analysis/missing_stability.csv")

In [7]:
print(len(interfaces_data))
print(len(pockets_data))
print(len(stability_data))
print(len(missing_data))

436012
436012
435875
237


In [8]:
435875 + 237

436112

In [6]:
stability_data

,Unnamed: 0.1,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,...,res_number,pK,state,position,uniprot_id,plddt,foldx_ddg_min,foldx_ddg_max,foldx_ddg_abs_max,foldx_ddg_abs_median
0,0,0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,...,2.0,NaN,undefined,2.0,P31946,74.30,-0.700058,0.130120,0.700058,0.096524
1,1,1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,...,6.0,NaN,undefined,6.0,P31946,95.62,-0.688997,0.643053,0.688997,0.278294
2,2,2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,...,21.0,11.100927,protonated,21.0,P31946,98.43,0.250080,3.602560,3.602560,2.686420
3,3,3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,...,32.0,NaN,undefined,32.0,P31946,98.24,-2.085020,19.898600,19.898600,3.844370
4,4,4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,...,39.0,NaN,undefined,39.0,P31946,97.05,-1.610100,2.446650,2.446650,1.516780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435870,435125,435258,PRAP1,PRAP1,Q96NZ9,10q26.3,Y113-ga,27914389,human,17.21,...,113.0,9.888425,protonated,113.0,Q96NZ9,87.73,-0.310755,0.859067,0.859067,0.516341
435871,435127,435260,PRG2,PRG2,P13727,11q12.1,T23-ga,27914778,human,25.21,...,23.0,NaN,undefined,23.0,P13727,39.53,-0.312937,0.381937,0.381937,0.134005
435872,435128,435261,PRG2,PRG2,P13727,11q12.1,S24-ga,27914781,human,25.21,...,24.0,NaN,undefined,24.0,P13727,45.31,-1.365570,0.171909,1.365570,0.471913
435873,435129,435262,PRG2,PRG2,P13727,11q12.1,T25-ga,27914784,human,25.21,...,25.0,NaN,undefined,25.0,P13727,40.84,-0.118950,0.782394,0.782394,0.100066


In [17]:
missing_data_no_duplicates = missing_data[~missing_data.SITE_GRP_ID.isin(stability_data.SITE_GRP_ID)]
missing_data_no_duplicates

,Unnamed: 0.1,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,...,res_number,pK,state,position,uniprot_id,plddt,foldx_ddg_min,foldx_ddg_max,foldx_ddg_abs_max,foldx_ddg_abs_median
28,272398,272506,YWHAB,14-3-3 beta,P31946,20q13.12,K5-ac,33347661,human,28.08,...,5.0,12.177229,protonated,5.0,P31946,95.10,-0.597878,1.59540,1.59540,0.744158
29,272399,272507,YWHAB,14-3-3 beta,P31946,20q13.12,K13-ac,36297548,human,28.08,...,13.0,11.797996,protonated,13.0,P31946,98.48,-0.852909,4.11965,4.11965,0.882116
30,272400,272508,YWHAB,14-3-3 beta,P31946,20q13.12,K51-ac,474927,human,28.08,...,51.0,10.448943,protonated,51.0,P31946,98.06,-1.387560,4.54226,4.54226,0.661567
31,272401,272509,YWHAB,14-3-3 beta,P31946,20q13.12,K70-ac,461819,human,28.08,...,70.0,10.624529,protonated,70.0,P31946,89.66,-0.058709,2.36513,2.36513,0.454511
32,272402,272510,YWHAB,14-3-3 beta,P31946,20q13.12,K77-ac,31068005,human,28.08,...,77.0,10.236019,protonated,77.0,P31946,89.25,-0.845906,1.71077,1.71077,0.668548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,297998,298115,SFN,14-3-3 sigma,P31947,1p36.11,K109-ub,15383715,human,27.77,...,109.0,10.921604,protonated,109.0,P31947,90.93,0.031068,2.67247,2.67247,1.468750
233,297999,298116,SFN,14-3-3 sigma,P31947,1p36.11,K122-ub,3933061,human,27.77,...,122.0,11.556411,protonated,122.0,P31947,98.63,-1.133220,6.29907,6.29907,1.714390
234,298000,298117,SFN,14-3-3 sigma,P31947,1p36.11,K124-ub,12381735,human,27.77,...,124.0,12.626807,protonated,124.0,P31947,98.79,-1.096620,7.02075,7.02075,1.466750
235,298001,298118,SFN,14-3-3 sigma,P31947,1p36.11,K140-ub,964495106,human,27.77,...,140.0,11.151598,protonated,140.0,P31947,93.77,0.570958,4.14529,4.14529,1.664520


In [20]:
full_stability_data = pd.concat([stability_data,missing_data_no_duplicates])
full_stability_data

,Unnamed: 0.1,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,...,res_number,pK,state,position,uniprot_id,plddt,foldx_ddg_min,foldx_ddg_max,foldx_ddg_abs_max,foldx_ddg_abs_median
0,0,0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,...,2.0,NaN,undefined,2.0,P31946,74.30,-0.700058,0.130120,0.700058,0.096524
1,1,1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,...,6.0,NaN,undefined,6.0,P31946,95.62,-0.688997,0.643053,0.688997,0.278294
2,2,2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,...,21.0,11.100927,protonated,21.0,P31946,98.43,0.250080,3.602560,3.602560,2.686420
3,3,3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,...,32.0,NaN,undefined,32.0,P31946,98.24,-2.085020,19.898600,19.898600,3.844370
4,4,4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,...,39.0,NaN,undefined,39.0,P31946,97.05,-1.610100,2.446650,2.446650,1.516780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,297998,298115,SFN,14-3-3 sigma,P31947,1p36.11,K109-ub,15383715,human,27.77,...,109.0,10.921604,protonated,109.0,P31947,90.93,0.031068,2.672470,2.672470,1.468750
233,297999,298116,SFN,14-3-3 sigma,P31947,1p36.11,K122-ub,3933061,human,27.77,...,122.0,11.556411,protonated,122.0,P31947,98.63,-1.133220,6.299070,6.299070,1.714390
234,298000,298117,SFN,14-3-3 sigma,P31947,1p36.11,K124-ub,12381735,human,27.77,...,124.0,12.626807,protonated,124.0,P31947,98.79,-1.096620,7.020750,7.020750,1.466750
235,298001,298118,SFN,14-3-3 sigma,P31947,1p36.11,K140-ub,964495106,human,27.77,...,140.0,11.151598,protonated,140.0,P31947,93.77,0.570958,4.145290,4.145290,1.664520


In [21]:
print(len(interfaces_data))
print(len(pockets_data))
print(len(stability_data))
print(len(full_stability_data))

436012
436012
435875
436012


In [22]:
full_stability_data.to_csv("/people/imal967/git_repos/pheno_analysis/results/merged_stability_FINAL.csv")